<a href="https://colab.research.google.com/github/UriNeri/ColabScan/blob/main/ColabScan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ColabScan !
Making it easier for non-bioinfos to search their data for RdRps.  
   WORK IN PROGRESS!!!   
   See the [README](https://github.com/UriNeri/ColabScan/blob/main/README.md) for more details

## Enviroment setup
First, we need to install the dependencies required.


In [ ]:
pip install multiprocessing
pip install pandas
pip install itables

mkdir bin
cd bin 
wget https://github.com/shenwei356/seqkit/releases/download/v2.4.0/seqkit_linux_amd64.tar.gz --quiet
tar -zxvf  seqkit_linux_amd64.tar.gz

wget http://eddylab.org/software/hmmer/hmmer.tar.gz --quiet
tar -zxvf hmmer.tar.gz
cd hmmer-3.3.2
./configure --silent
make --silent
make install

# Binary workaround 
Becuase for the life of me it seems variables are lost between blocks except for the python codeblocks/kernal, we'll add the above installed binaries directly from Python.  
If it was up to me, this was a single bash script, but notebook helps people...

In [20]:
import os as os

os.environ['PATH'] = os.environ['PATH']+":"+os.getcwd()+"/bin/"
os.environ['PATH'] = os.environ['PATH']+":"+os.getcwd()+"/bin/hmmer-3.3.2/src/"
os.environ['PATH'] = os.environ['PATH']+":"+os.getcwd()+"/bin/parallel-20230622/src/"
print(os.environ['PATH'])
os.environ['USE_parallel'] = "FALSE"

/home/neri/work/miniconda3/bin:/home/neri/work/miniconda3/condabin:/home/neri/bin:/home/neri/Documents/mmseqs/bin:/home/neri/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/bin:/var/lib/flatpak/exports/bin:/usr/lib/jvm/default/bin:/usr/bin/site_perl:/usr/bin/vendor_perl:/usr/bin/core_perl:/home/neri/stl/prefix:/home/neri/work/ColabScan/bin/:/home/neri/work/ColabScan/bin/hmmer-3.3.2/src/:/home/neri/work/ColabScan/bin/parallel-20230622/src/


## Multithreading (optional)
It is possible to speed things up by parallelizing some tasks or suppling `hmmsearch` with more cores. This only applies to running the notebook locally. If you want to go ahead with this, run the following code block, which will set an enviromental variables `THREADS` which would use all available cores minus one (you can manually change this if you want).


In [21]:
from multiprocessing import cpu_count
import os as os
os.environ['THREADS'] = str(cpu_count() - 1)
os.environ['USE_parallel'] = "TRUE"

In [22]:
%%shell
wget https://github.com/shenwei356/seqkit/releases/download/v2.4.0/seqkit_linux_amd64.tar.gz --quiet
tar -zxvf  seqkit_linux_amd64.tar.gz

wget http://eddylab.org/software/hmmer/hmmer.tar.gz --quiet
tar -zxvf hmmer.tar.gz
cd hmmer-3.3.2
./configure
make
make install
# workaround for permission trouble or if you are running locally, aliasing to local path
alias hmmbuild='$(pwd)/src/hmmbuild'
alias hmmsearch='$(pwd)/src/hmmsearch'
cd ..
alias seqkit='$(pwd)/seqkit'
pip install itables
pip install pandas


UsageError: Cell magic `%%shell` not found.


## Data fetching
Next we're gonna fetch the pHMMs/DBs directly from the ColabScan github repo. For this purpose we'll assume you are running this notebook from a local version of the github repo. If you haven't already downloaded the rest of the repo, the next block does just that.


In [10]:
%%bash
# git clone --recursive https://github.com/UriNeri/ColabScan.git
bzip2 -d ./DBs/*bz2

## Select your sequences!
Now you'll need to set the path to the fasta file you want to scan. The path needs to be absoulte (starting at root `/`).

In [23]:
os.environ['Input_file'] = "/home/neri/work/ColabScan/selected_gencode_6ers.fasta"

## Set cutoffs
You can limit the results and some internal steps of `hmmsearch` so the output only contains matchs with passing pre-set thersholds. The values below are reletavily relaxed. 

In [24]:
import os as os
os.environ['score'] = str(20)
os.environ['E_value'] = str(0.0001)


## HMMsearching  
Next we are going to do start an hmmsearch against the different DBs.

In [27]:
%%bash
HMM_DBs=($(pwd)/DBs/Colab_RdRp_hmmer_3.2.2.hmm $(pwd)/DBs/NeoRdRp-HMM.v1.1.hmm)
tmpstamp=test_"$(date "+%F%T")"
mkdir $tmpstamp
cd $tmpstamp

ini_name=$(basename $Input_file ".fasta")
seqkit translate -x -F --clean -w 20000 -f 6 $Input_file --id-regexp "(\*)" > ./"$ini_name".faa


echo "input file is" $ini_name
echo "Maximal E-value file is"  $eval
echo "Minimal score is" $score

for hmmdb in ${HMM_DBs[*]}; do
hmmdb_name=$(basename $hmmdb .hmm)
echo "Starting hmmsearch of " $ini_name " vs " $hmmdb_name
hmmsearch --cpu $THREADS -E $eval --domE $eval --incdomE $eval --incE $eval -o "$hmmdb_name"_hmmsearch_vs_"$ini_name".tsv --domtblout domtblout_"$hmmdb_name"_hmmsearch_vs_"$ini_name".tsv --tblout tabular_"$hmmdb_name"_hmmsearch_vs_"$ini_name".tsv $hmmdb ./"$ini_name".faa
sed -i '/^#/d' domtblout_"$hmmdb_name"_hmmsearch_vs_"$ini_name".tsv
# organize the output neat-ly-er
awk '{print $1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7"\t"$8"\t"$9"\t"$10"\t"$11"\t"$12"\t"$13"\t"$14"\t"$15"\t"$16"\t"$17"\t"$18"\t"$19"\t"$20"\t"$21"\t"$22"\t"$23"\t"$24}' domtblout_"$hmmdb_name"_hmmsearch_vs_"$ini_name".tsv >>awked_domtblout_"$hmmdb_name"_hmmsearch_vs_"$ini_name".tsv #Legacy - don't chage to OFS because of trailing sep issue with parser
done

# Merge and write the headers
echo "target_name	target_accession	qL	query_name	query_accession	pL	E-value_fullseq	score_fullseq	bias_fullseq	#	of	c-Evalue	i-Evalue	bias_thisdomain	score_thisdomain	p1	p2	q1	q2	env_from	env_to	acc	description_of_target" > catted_awked_vs_"$ini_name".tsv
cat awked_domtblout_* >> catted_awked_vs_"$ini_name".tsv


input file is selected_gencode_6ers
Maximal E-value file is 0.0001
Minimal score is 20
Starting hmmsearch of  selected_gencode_6ers  vs  Colab_RdRp_hmmer_3.2.2
Starting hmmsearch of  selected_gencode_6ers  vs  NeoRdRp-HMM.v1.1


In [29]:
import pandas as pd
import glob as glob
import itables as it
Hits = pd.read_table(str(glob.glob("./test*/catted_awked_vs_*")[0]), sep='\t',index_col=False)
it.show(Hits,maxBytes=0)

